# Scripts Mosaicos 90's

In [1]:
import ee
import geemap
import os
import folium

In [2]:
Map = geemap.Map()
Map2 = geemap.Map()
Map3 = geemap.Map()

## Área de Mosaicos

### Patagonia

In [3]:
work_dir = os.path.expanduser('../../Data/shapes/')
patagonia = os.path.join(work_dir, 'patagonia_nueva.shp')
patagonia_grande = geemap.shp_to_ee(patagonia)
Map.addLayer(patagonia_grande, {}, 'Patagonia')
Map

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [4]:
work_dir = os.path.expanduser('../../Data/shapes/')
norte = os.path.join(work_dir, 'area_norte.shp')
patagonia_norte = geemap.shp_to_ee(norte)
Map.addLayer(patagonia_norte, {}, 'norte')

In [5]:
work_dir = os.path.expanduser('../../Data/shapes/')
centro = os.path.join(work_dir, 'area_centro.shp')
patagonia_centro = geemap.shp_to_ee(centro)
Map.addLayer(patagonia_centro, {}, 'centro')

In [6]:
work_dir = os.path.expanduser('../../Data/shapes/')
sur = os.path.join(work_dir, 'area_sur.shp')
patagonia_sur = geemap.shp_to_ee(sur)
Map.addLayer(patagonia_grande, {}, 'sur')

## Funciones

In [7]:
def addNDVI(image): 
    ndvi = image.normalizedDifference(['B4', 'B3']).rename('NDVI')
    return image.addBands(ndvi)

In [8]:
def addNDWI(image):
    ndwi = image.normalizedDifference(['B2','B4']).rename('NDWI')
    return image.addBands(ndwi)

In [9]:
def addNDGI(image):
    ndgi = image.normalizedDifference(['B2','B3']).rename('NDGI')
    return image.addBands(ndgi)

In [10]:
def addDate(image):
    img_date = ee.Date(image.date())
    img_date = ee.Number.parse(img_date.format('YYYYMMdd'))
    return image.addBands(ee.Image(img_date).rename('date').toInt())

In [11]:
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer

In [12]:
def Cloud50(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(50))  

In [13]:
def Cloud30(image):
    scored = ee.Algorithms.Landsat.simpleCloudScore(image)
    return image.updateMask(scored.select(['cloud']).lt(30)) 

In [14]:
def Agua(image):
    return image.updateMask(ndwi.lte(-0.1))

In [15]:
def Agua1996(image):
    return image.updateMask(ndwi1996.gte(0.1))

In [16]:
def Agua1997(image):
    return image.updateMask(ndwi1997.gte(0))

In [17]:
def Agua1998(image):
    return image.updateMask(ndwi1998.gte(0))

In [18]:
def Ndvi1996(image):
    return image.updateMask(ndvi1996.gt(0.089))

In [19]:
def Ndvi1997(image):
    return image.updateMask(ndvi1997.lt(0.2))

In [20]:
def Ndvi1998(image):
    return image.updateMask(ndvi1998.lt(0.2))

In [21]:
def Ndvi2(image):
    return image.updateMask(ndvi.gte(0.2))

### Funciones SUR

In [22]:
def Ndvi1996_sur(image):
    return image.updateMask(ndvisur.gt(0.04))

In [23]:
def contra_Ndvi1996_sur(image):
    return image.updateMask(ndvisur.lte(0.04))

In [24]:
def Agua1996_sur(image):
    return image.updateMask(ndwisur.gte(0.3))

In [25]:
def contra_Agua1996_sur(image):
    return image.updateMask(ndwisur.lt(0.3))

## Colecciones

### Verano 1996

In [26]:
start_date = '1996-10-01'
end_date = '1997-03-31'

l5_1996 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [27]:
vizParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.5,
  'gamma': [0.95, 1.1, 1]
}

In [28]:
coleccion1996 = l5_1996.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

#### Verano Sur 1996-1999

In [29]:
start_date = '1996-10-01'
end_date = '2001-03-31'

l5_sur = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [30]:
coleccionsur = l5_sur.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

### Verano 1997

In [31]:
start_date = '1997-10-01'
end_date = '1998-03-31'

l5_1997 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [32]:
coleccion1997 = l5_1997.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

### Verano 1998

In [33]:
start_date = '1998-10-01'
end_date = '1999-03-31'

l5_1998 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [34]:
coleccion1998 = l5_1998.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

## Quality Mosaic NDVI

In [35]:
NDVI_1996 = coleccion1996.qualityMosaic('NDVI')
NDVI_1997 = coleccion1997.qualityMosaic('NDVI')
NDVI_1998 = coleccion1998.qualityMosaic('NDVI')

In [36]:
NDVI_sur = coleccionsur.qualityMosaic('NDVI')

In [37]:
visParams = {
  'bands': ['B4', 'B3', 'B2'],
  'min': 0,
  'max': 0.4,
}
Map.addLayer(NDVI_1996, visParams, 'Greenest 1996')
Map.addLayer(NDVI_1997, visParams, 'Greenest 1997')
Map.addLayer(NDVI_1998, visParams, 'Greenest 1998')
Map.addLayer(NDVI_sur, visParams, 'Greenest sur 1996-1999')
Map.centerObject(patagonia_grande, 6)
Map

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position', 'transparent…

In [38]:
ndvi1996 = NDVI_1996.select('NDVI')
ndvi1997 = NDVI_1997.select('NDVI')
ndvi1998 = NDVI_1998.select('NDVI')
ndvisur = NDVI_sur.select('NDVI')
palette = ['#d73027', '#f46d43', '#fdae61', '#fee08b', '#d9ef8b', '#a6d96a', '#66bd63', '#1a9850']

In [39]:
Map.addLayer(ndvi1996, {'palette': palette}, 'NDVI_1996')
Map.addLayer(ndvi1997, {'palette': palette}, 'NDVI_1997')
Map.addLayer(ndvi1998, {'palette': palette}, 'NDVI_1998')
Map.addLayer(ndvisur,{'palette': palette}, 'NDVI_sur 1996-1999')
Map

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position', 'transparent…

## Agua

### Verano 1996

In [40]:
aquast_1996 = coleccion1996.qualityMosaic('NDWI')
aquast_sur = coleccionsur.qualityMosaic('NDWI')

### Verano 1997

In [41]:
aquast_1997 = coleccion1997.qualityMosaic('NDWI')

### Verano 1998

In [42]:
aquast_1998 = coleccion1998.qualityMosaic('NDWI')

### Quality Mosaic NDWI

In [43]:
visParams = {
  'bands': ['B4', 'B3', 'B1'],
  'min': 0,
  'max': 0.4,
    'gamma': [0.95, 1.1, 1],
}

Map2.addLayer(aquast_1996, visParams, 'Agua 1996')
Map2.addLayer(aquast_1997, visParams, 'Agua 1997')
Map2.addLayer(aquast_1998, visParams, 'Agua 1998')
Map2.addLayer(aquast_sur, visParams, 'Agua sur 1996-1999')
Map2.centerObject(patagonia_grande, 6)
Map2

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position', 'transparent…

In [44]:
ndwi1996 = aquast_1996.select('NDWI')
ndwi_viz = ['00FFFF', '0000FF']
ndwi1997 = aquast_1997.select('NDWI')
ndwi1998 = aquast_1998.select('NDWI')
ndwisur = aquast_sur.select('NDWI')
Map2.addLayer(ndwi1996, {'palette': ndwi_viz}, 'NDWI_1996')
Map2.addLayer(ndwi1997, {'palette': ndwi_viz}, 'NDWI_1997')
Map2.addLayer(ndwi1998, {'palette': ndwi_viz}, 'NDWI_1998')
Map2.addLayer(ndwisur, {'palette': ndwi_viz}, 'NDWI_sur 1996-1999')
Map2

Map(center=[-47.89147413528022, -68.82549165231002], controls=(WidgetControl(options=['position', 'transparent…

In [45]:
ndwi_rgb = ndwi1996.visualize(**{
    'min': 0.5,
    'max':1,
    'palette': ['00FFFF', '0000FF']
})

In [46]:
ndwi_mask = ndwi1996.updateMask(ndwi1996.gte(0.4))

In [47]:
map_ndwi_mask = folium.Map(location = [-45, -73], zoom_start = 5)
map_ndwi_mask.add_ee_layer(ndwi_mask, {} , 'NDWI sin tierra')
display(map_ndwi_mask)

In [48]:
ndwi_mask1996 = ndwi1996.updateMask(ndwi1996.gte(0.4))

In [49]:
map_ndwi_mask1996 = folium.Map(location = [-45, -73], zoom_start = 5)
map_ndwi_mask1996.add_ee_layer(ndwi_mask1996, {}, 'NDWI sin tierra')
display(map_ndwi_mask1996)

## Máscaras

### Verano 1996

#### Nubes 50

In [50]:
coleccion96 = coleccion1996.map(Cloud50)
greenest96 = coleccion96.qualityMosaic('NDVI')
map_1996 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1996.add_ee_layer(greenest96, vizParams, 'Mapa sin Nubes')
display(map_1996)

#### Nubes 30

In [51]:
coleccion97 = coleccion1997.map(Cloud30)
greenest97 = coleccion97.qualityMosaic('NDVI')
map_1997 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1997.add_ee_layer(greenest97, vizParams, 'Mapa sin Nubes')
display(map_1997)

#### Nubes Sur

In [52]:
coleccion_sur = coleccionsur.map(Cloud30)
greenestsur = coleccion_sur.qualityMosaic('NDVI')
map_sur = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur.add_ee_layer(greenestsur, vizParams, 'Mapa sin Nubes')
display(map_sur)

In [53]:
coleccionsur2 = coleccionsur.map(Cloud50)
greenestsur2 = coleccionsur2.qualityMosaic('NDVI')
map_sur2 = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur2.add_ee_layer(greenestsur2, vizParams, 'Mapa sin Nubes')
display(map_sur2)

In [54]:
coleccion_sur = coleccionsur.map(Cloud30)
coleccion_sur = coleccion_sur.map(Agua1996_sur)
aquastsur = coleccion_sur.qualityMosaic('NDWI')
map_sur = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur.add_ee_layer(aquastsur, vizParams, 'Mapa sin Nubes')
display(map_sur)

In [55]:
coleccion_sur = coleccionsur.map(Cloud30)
coleccion_sur = coleccion_sur.map(Ndvi1996_sur)
tierrasur = coleccion_sur.qualityMosaic('NDVI')
map_sur = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur.add_ee_layer(tierrasur, vizParams, 'Mapa sin Nubes')
display(map_sur)

#### Filtro de agua

In [56]:
coleccion96_ = coleccion1996.map(Cloud50)
coleccion96_ = coleccion96_.map(Agua1996)
greenest96_ = coleccion96_.qualityMosaic('NDVI')
map_1996_ = folium.Map(location = [-45, -73], zoom_start = 5)
map_1996_.add_ee_layer(greenest96_, vizParams, 'Mapa sin Nubes')
display(map_1996_)

In [57]:
agua_1996 = coleccion1996.map(Cloud50)
agua_1996 = agua_1996.map(Agua1996)
aquast96_ = agua_1996.qualityMosaic('NDWI')
map_agua1996 = folium.Map(location = [-45, -73], zoom_start = 5)
map_agua1996.add_ee_layer(aquast96_, vizParams, 'Mapa sin Nubes')
display(map_agua1996)

In [58]:
coleccion96_2 = coleccion1996.map(Cloud50)
greenest96_2 = coleccion96_2.qualityMosaic('NDVI')
map_1996_2 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1996_2.add_ee_layer(greenest96_2, vizParams, 'Mapa sin Nubes')
display(map_1996_2)

#### Filtro NDVI

In [59]:
coleccion96_3 = coleccion1996.map(Cloud50)
coleccion96_3 = coleccion96_3.map(Ndvi1996)
greenest96_3 = coleccion96_3.qualityMosaic('NDVI')
map_1996_3 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1996_3.add_ee_layer(greenest96_3, vizParams, 'Mapa sin Nubes')
display(map_1996_3)

In [60]:
coleccion_sur2 = coleccionsur.map(Cloud30)
coleccion_sur2 = coleccion_sur2.map(Ndvi1996_sur)
coleccion_sur2 = coleccion_sur2.map(Agua1996_sur)
greenest_sur2 = coleccion_sur2.qualityMosaic('NDVI')
map_sur2 = folium.Map(location = [-45, -73], zoom_start = 5)
map_sur2.add_ee_layer(greenest_sur2, vizParams, 'Mapa sin Nubes')
display(map_sur2)

### Verano 1997

In [61]:
coleccion97 = coleccion1997.map(Cloud50)

greenest97 = coleccion97.qualityMosaic('NDVI')
map_1997 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1997.add_ee_layer(greenest97, vizParams, 'Mapa sin nubes 97')
display(map_1997)

In [62]:
agua_1997 = coleccion1997.map(Agua1997)
aquast97_ = agua_1997.qualityMosaic('NDWI')
map_agua1997 = folium.Map(location = [-45, -73], zoom_start = 5)
map_agua1997.add_ee_layer(aquast97_, vizParams, 'Mapa sin Nubes 97')
display(map_agua1997)

### Verano 1998

In [63]:
coleccion98 = coleccion1998.map(Cloud50)
# coleccion2019 = coleccion2019.map(Agua)
greenest98 = coleccion98.qualityMosaic('NDVI')
map_1998 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1998.add_ee_layer(greenest98, vizParams, 'Mapa sin nubes')
display(map_1998)

### Verano 1995

In [64]:
start_date = '1995-10-01'
end_date = '1996-03-31'

l5_1995 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)


In [65]:
coleccion1995 = l5_1995.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [66]:
coleccion95 = coleccion1995.map(Cloud30)
# coleccion2019 = coleccion2019.map(Agua)
greenest95 = coleccion95.qualityMosaic('NDVI')
map_1995 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1995.add_ee_layer(greenest95, vizParams, 'Mapa sin nubes')
display(map_1995)


### Verano 1994

In [67]:
start_date = '1994-10-01'
end_date = '1995-03-31'

l5_1994 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [68]:
coleccion1994 = l5_1994.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [69]:
coleccion94 = coleccion1994.map(Cloud30)
# coleccion2019 = coleccion2019.map(Agua)
greenest94 = coleccion94.qualityMosaic('NDVI')
map_1994 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1994.add_ee_layer(greenest94, vizParams, 'Mapa sin nubes')
display(map_1994)

### Verano 1993

In [70]:
start_date = '1993-10-01'
end_date = '1994-03-31'

l5_1993 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)



In [71]:
coleccion1993 = l5_1993.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [72]:
coleccion93 = coleccion1993.map(Cloud30)
# coleccion2019 = coleccion2019.map(Agua)
greenest93 = coleccion93.qualityMosaic('NDVI')
map_1993 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1993.add_ee_layer(greenest93, vizParams, 'Mapa sin nubes')
display(map_1993)

### Verano 1992

In [73]:
start_date = '1992-10-01'
end_date = '1993-03-31'

l5_1992 = ee.ImageCollection('LANDSAT/LT05/C01/T1_TOA') \
    .filterBounds(patagonia_grande) \
    .filterDate(start_date, end_date)

In [74]:
coleccion1992 = l5_1992.map(addNDVI).map(addNDWI).map(addNDGI).map(addDate)

In [75]:
coleccion92 = coleccion1992.map(Cloud30)
# coleccion2019 = coleccion2019.map(Agua)
greenest92 = coleccion92.qualityMosaic('NDVI')
map_1992 = folium.Map(location = [-45, -73], zoom_start = 5)
map_1992.add_ee_layer(greenest92, vizParams, 'Mapa sin nubes')
display(map_1992)

## Mosaicos

In [76]:
Map3.addLayer(ndvi1996, {'palette': palette}, 'NDVI_1996')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [77]:
mosaic = ee.ImageCollection([greenestsur,aquastsur]).mosaic()

In [78]:
Map3.addLayer(mosaic, vizParams, 'Mosaico')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [79]:
mosaic2 = ee.ImageCollection([mosaic, tierrasur]).mosaic()

In [80]:
Map3.addLayer(mosaic2, vizParams, 'Mosaico2')
Map3

Map(center=[40, -100], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [81]:
# mosaic3 = ee.ImageCollection([mosaic2, greenest97]).mosaic()

In [82]:
# Map3.addLayer(mosaic3, vizParams, 'Mosaico3')
# Map3

In [83]:
# mosaic4 = ee.ImageCollection([mosaic3, greenest_sur2]).mosaic()

In [84]:
# Map3.addLayer(mosaic4, vizParams, 'Mosaico 4')
# Map3

In [85]:
# mosaic5 = ee.ImageCollection([mosaic4,aquast97_]).mosaic()

In [86]:
# Map3.addLayer(mosaic5, vizParams, 'Mosaico 5')
# Map3

## Charts